In [1]:
# Lazimi kitabxanalari yukleyirik
import pandas as pd
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC

C:\Users\qubadlim\AppData\Local\Temp\ipykernel_6876\2113332174.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# ilk once chrome driverin en son surumu yuklenir daha sonra Chrome webdriveri yaradilir
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [3]:
# sayt linki daxil edilir daha sonra driver.get() metoduna link gonderilerek chrome browser acilir
base_url = 'https://turbo.az/autos'
driver.get(base_url)

In [4]:
# Turbo.az saytinda yerlesdirilmis butun elan linklerini saxlamaq ucun ads_links deyiskeni yaradilir
ads_links = []

# seyfe sayini bilmediyimiz ucun sonsuz dovr istifade edirik
while True:
    # linkleri cekmek ucun elanlari icinde saxlayan parent elementi tapiriq
    ads_container = driver.find_element(By.CSS_SELECTOR, '.products-container')

    # product-conatineri altindaki butun elanlari selenium elemnt listi kimi ads_elements e menimsedirik
    ads_elements = ads_container.find_elements(By.CSS_SELECTOR, '.products-i .products-i__link')

    # her bir elan elementi uzerinde gezib onun 'href' atributundaki deyer cekib ads_links listine elave edirik
    for element in ads_elements:
        ads_links.append(element.get_attribute('href'))

    # diger seyfelerdeki elan linklerini goturmek ucunde novbeti seyfeye kecide imkan veren 'Next' buttonu tapiriq ve click() scriptini ise saliriq
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, 'a[rel="next"]')
        driver.execute_script("arguments[0].click();", next_button)

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[rel="next"]')))
    except Exception as e:
        # her hansi xeta yaranarsa ekrana yazdirilir ve eger 'Next' butonu tapilmazsa dovr dayandirilir
        print(e)
        break


print(len(ads_links))

Message: 
Stacktrace:
	GetHandleVerifier [0x008F1673+52979]
	(No symbol) [0x00877961]
	(No symbol) [0x0075DD3D]
	(No symbol) [0x00795FBB]
	(No symbol) [0x007960FB]
	(No symbol) [0x007CCF92]
	(No symbol) [0x007B4534]
	(No symbol) [0x007CB3FE]
	(No symbol) [0x007B4286]
	(No symbol) [0x0078C063]
	(No symbol) [0x0078CECD]
	GetHandleVerifier [0x00C08D83+3294723]
	GetHandleVerifier [0x00C46CC2+3548482]
	GetHandleVerifier [0x00C41C9C+3527964]
	GetHandleVerifier [0x0098870E+671630]
	(No symbol) [0x00881EB4]
	(No symbol) [0x0087D808]
	(No symbol) [0x0087D92D]
	(No symbol) [0x0086F7E0]
	BaseThreadInitThunk [0x763EFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77757B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77757AEE+238]

14940


In [5]:
# butun elanlarin melumatlarini saxlamaq ucun bos list yaradılır
all_ads_infos =[]

# Elanlardaki melumatlari cekmek ucun funksiya yazilir. Parametr olarak ads_links de saxladigimiz linkler oturulur
def get_ad_infos(url):

    # her bir linki webdriverde aciriq
    driver.get(url)

    # Elan melumatlarini saxlamaq ucun dictionary yaradiriq
    ad_infos = {}

    # Elan seyfesinde .product container elementinin yerlesmesini gozduyuruk
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.product')))

    # Melumatlari cekmek ucun Html faylinda lazimi elementleri secirik
    product_container = driver.find_element(By.CSS_SELECTOR, '.product')
    product_sidebar_container = product_container.find_element(By.CSS_SELECTOR, '.product-sidebar-container .product-sidebar')
    product_photos_container = product_container.find_element(By.CSS_SELECTOR, '.product-photos')
    product_desc_container = product_container.find_element(By.CSS_SELECTOR, '.product-description__content')

    # Avtomabil xususiyyetlerinin qeyd olundurugu elementleri list olaraq properties deyiskenine menimsedirik
    properties = product_container.find_elements(By.CSS_SELECTOR, '.product-properties__i')

    # properties listindeki elementler uzerinde bir bir gezib avtomabilin xususiyyetlerinin adini ve deyerini uygun deyisenlere menimsedirik
    # qeyd: burada bir cox xusisyyetin html strukturu eyni oldugu ucun bu cur yanasma tetbiq olunub
    for prop in properties:
        prop_name = prop.find_element(By.CSS_SELECTOR, '.product-properties__i-name').text
        prop_value = prop.find_element(By.CSS_SELECTOR, '.product-properties__i-value').text
 
        # xususiyyetin adini key, deyerini ise value olaraq ad_infos sozluyune elave edirik
        ad_infos[prop_name] = prop_value

    # avtomabilin asagidaki xususiyyetlerinin ozunemexsus html strukturu oldugu ucun her birine individual yanasaraq melumatlar cekilir
    ad_infos['price'] = product_sidebar_container.find_element(By.CSS_SELECTOR, '.product-price .product-price__i').text

    # elan seyfelerinde elan sahibinin adi iki ferqli strukturda ola biler. elan ya agentlik terefinden yada fiziki sexs terefinden verile biler
    # bu serti nezere almaq ucun asagidaki kimi bir yanasma tetbiq olunmusudur
    seller_elements = product_sidebar_container.find_elements(By.CSS_SELECTOR, '.product-owner .product-owner__info-name, .product-shop__owner-name')

    if seller_elements:
        ad_infos['seller'] = seller_elements[0].text
    else:
        ad_infos['seller'] = None

    ad_infos['seller_comment'] = product_desc_container.find_element(By.TAG_NAME, 'p').text
    ad_infos['img_url'] = product_photos_container.find_elements(By.CSS_SELECTOR, 'img')[1].get_attribute('src')

    # elan melumatlari sozluk kimi qaytarilir
    return ad_infos


In [6]:
# ads_links listindeki butun elan linkleri uzerinde gezib get_ad_infos() funksiyasi ile datalari cekirik. Qeyd: elanlarin umumi sayi 14940 ededir.
for link in ads_links:
    ad_info = get_ad_infos(link)
    all_ads_infos.append(ad_info)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".product-description__content"}
  (Session info: chrome=121.0.6167.161); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x008F1673+52979]
	(No symbol) [0x00877961]
	(No symbol) [0x0075DD3D]
	(No symbol) [0x00795FBB]
	(No symbol) [0x007960FB]
	(No symbol) [0x0078DAD1]
	(No symbol) [0x007B4534]
	(No symbol) [0x0078DA4E]
	(No symbol) [0x007B4784]
	(No symbol) [0x007CB3FE]
	(No symbol) [0x007B4286]
	(No symbol) [0x0078C063]
	(No symbol) [0x0078CECD]
	GetHandleVerifier [0x00C08D83+3294723]
	GetHandleVerifier [0x00C46CC2+3548482]
	GetHandleVerifier [0x00C41C9C+3527964]
	GetHandleVerifier [0x0098870E+671630]
	(No symbol) [0x00881EB4]
	(No symbol) [0x0087D808]
	(No symbol) [0x0087D92D]
	(No symbol) [0x0086F7E0]
	BaseThreadInitThunk [0x763EFCC9+25]
	RtlGet

In [7]:
# all_ads_infos listindeki butun elementleri pandas DataFrameine ceviririk
df = pd.DataFrame(all_ads_infos)

# Butun sutunlarin adlarini ingilis dilindeki qarsiliqlari ile deyisdirik
df = df.rename(columns={
    'Şəhər' : 'city',
    'Marka' : 'brand',
    'Model' : 'model',
    'Buraxılış ili' : 'production_year',
    'Ban növü' : 'ban_type',
    'Rəng' : 'color',
    'Mühərrik' : 'engine',
    'Yürüş' : 'mileage',
    'Sürətlər qutusu' : 'gear_box',
    'Ötürücü' : 'traction',
    'Yeni' : 'new',
    'Vəziyyəti' : 'the_situation',
    'Hansı bazar üçün yığılıb' : 'market',
    'Yerlərin sayı' : 'seats',
    'Sahiblər' : 'owners',
})

In [8]:
# DataFramein ilk 5 setirini ekrana cixaririq
df.shape

(8, 19)

In [ ]:
# DataFrame 'turbo_az_dataset.csv' fayli kimi save edirik
df.to_csv('../DataSets/turbo_az_dataset.csv')

In [ ]:
# DataFrame den butun sekil linlerini alib list kimi qaytaririq
img_url_list = df['img_url'].tolist()

# Sekilleri save etmek ucun fayl yolunu qeyd edirik
photo_folder = r'C:\Users\qubadlim\Desktop\Python\DataSets\car_photos'

# Eger folder yoxdursa yaradiriq
if not os.path.exists(photo_folder):
    os.makedirs(photo_folder)

# Her bir sekili endirib save edirik
for i, img_url in enumerate(img_url_list):
    response = requests.get(img_url)
    if response.status_code == 200:
        with open(os.path.join(photo_folder, f'car_photo_{i+1}.jpg'), 'wb') as f:
            f.write(response.content)